Derived quantities may be computed and written to file during the course of the simulation, or alternatively during the postprocessing stage. The advantage of computing them during the course of the simulation is ... using for stoppers, flagging numerical instabilities ...

In [1]:
from ufl import inner, grad
from dolfinx.fem import FunctionSpace, VectorFunctionSpace
from lucifex.fem import StaticConstant, StaticFunction
from lucifex.fdm import FunctionSeries, ConstantSeries
from lucifex.mesh import create_rectangle
from lucifex.solver import ds_solver, dS_solver
from lucifex.utils import set_fem_function


Lx = 2.0
Ly = 1.0
mesh = create_rectangle(Lx, Ly, 50, 50, name="LxLy")

fs_vector = VectorFunctionSpace(mesh, ('P', 1), 2)
u = StaticFunction(fs_vector)
set_fem_function(u, (1.0, -1.0))

fs_scalar = FunctionSpace(mesh, ('P', 1))
c = StaticFunction(fs_scalar)
set_fem_function(c, lambda x: x[1]**2)

flux = StaticConstant(mesh, name="Fh0")
flux2 = StaticConstant(mesh, name="Ftop")
flux_expression = lambda u, c: inner(u, grad(c))


h = 0.6
flux_solver = dS_solver(flux_expression, lambda x: x[1] - h)(flux, u('+'), c('+'))
flux_solver.solve()
print(flux.value, -2*Lx*h)

flux2_solver = ds_solver(flux_expression, lambda x: x[1] - Ly)(flux2, u, c)
flux2_solver.solve()
print(flux2.value, -2*Lx*Ly)


-2.359999999999998 -2.4
-3.9600000000000013 -4.0


In [3]:
flux2_solver.measure

Measure('exterior_facet', subdomain_id=0, domain=Mesh(VectorElement(Basix element (P, quadrilateral, 1, gll_warped, unset, False), 2), 0), subdomain_data=<dolfinx.mesh.MeshTagsMetaClass object at 0x13fe35ee0>)